In [0]:
from __future__ import division
import nltk, re, pprint

from urllib.request import urlopen

from nltk import book

from nltk.corpus import gutenberg
from nltk.corpus import brown
from nltk.corpus import wordnet as wn
from nltk.corpus import abc
import random
from nltk.corpus import names
from nltk.classify import apply_features
from nltk.corpus import movie_reviews
from nltk.tag import hmm
from nltk.corpus import ppattach

exercise 2.  Design at least 5 features and report what these features capture. Additionally, use three classifiers, namely, nltk.NaiveBayesClassifier, nltk.DecisionTreeClassifier,  nltk.MaxentClassifier. Compare the performance of the three classifiers by analyzing the accuracy. Report the accuracy of each classifier built using all of the features that you designed.

In [45]:
def exercise2():
    labeled_names = ([(name, 'male') for name in names.words('male.txt')] + [(name, 'female') for name in names.words('female.txt')])
    test, devtest, training = labeled_names[:500], labeled_names[500:1000], labeled_names[1000:]

    def gender_features(name):
        features = {}
        features["first_letter"] = name[0].lower()  # feature1
        features["last_letter"] = name[-1].lower()  # feature2
        for letter in 'abcdefghijklmnopqrstuvwxyz':
            features["count({})".format(letter)] = name.lower().count(letter)  # feature3
            features["has({})".format(letter)] = (letter in name.lower())  # feature4
        features['vowels_len'] = len([word for word in name if word in 'aeiou'])  # feature5
        return features

    train_set = [(gender_features(n), g) for (n, g) in training]
    devtest_set = [(gender_features(n), g) for (n, g) in devtest]
    test_set = [(gender_features(n), gender) for (n, gender) in test]

    print("Note: MaxentClassifier will train, which taked a while to run the program.")
    NaiveBayes = nltk.NaiveBayesClassifier.train(train_set)
    DecisionTree = nltk.DecisionTreeClassifier.train(train_set)
    Maxentropy = nltk.MaxentClassifier.train(train_set)

    print("NaiveByesClassifier:")
    print("Performance, dev-test set: ",nltk.classify.accuracy(NaiveBayes,devtest_set))
    print("Performance on test_set:",nltk.classify.accuracy(NaiveBayes,test_set))
    print("")
    print("DecisionTree:")
    print("Performance, dev-test set: ", nltk.classify.accuracy(DecisionTree, devtest_set))
    print("Performance on test_set:", nltk.classify.accuracy(DecisionTree, test_set))
    print("")
    print("Maxentropy:")
    print("Performance, dev-test set: ", nltk.classify.accuracy(Maxentropy, devtest_set))
    print("Performance on test_set:", nltk.classify.accuracy(Maxentropy, test_set))


exercise2()    

Note: MaxentClassifier will train, which taked a while to run the program.
  ==> Training (100 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.280
             2          -0.50151        0.720
             3          -0.48984        0.720
             4          -0.47877        0.720
             5          -0.46830        0.720
             6          -0.45842        0.721
             7          -0.44913        0.723
             8          -0.44039        0.727
             9          -0.43217        0.734
            10          -0.42446        0.743
            11          -0.41721        0.751
            12          -0.41040        0.759
            13          -0.40400        0.767
            14          -0.39797        0.772
            15          -0.39229        0.778
            16          -0.38694        0.783
            17          -0.38188        0.788
            18    

exercise 4. pick any 5 features out of the computed 30 and describe their relevance.

In [0]:
def exercise4():
    documents = [(list(movie_reviews.words(fileid)), category) for category in movie_reviews.categories() for fileid in movie_reviews.fileids(category)]
    all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())
    word_features = list(all_words)

    def document_features(document):
        document_words = set(document)
        features = {}
        for word in word_features:
            features['contains({})'.format(word)] = (word in document_words)
        return features
    print("This part will take a while to implement")
    featuresets = [(document_features(d), c) for (d, c) in documents]
    train_set, test_set = featuresets[100:], featuresets[:100]
    classifier = nltk.NaiveBayesClassifier.train(train_set)
    print("The accuracy of classifier on test set is: ")
    print(nltk.classify.accuracy(classifier, test_set))
    classifier.show_most_informative_features(30)
    print("The top five feature is: ")
    classifier.show_most_informative_features(5)

exercise4()

exercise 7.  Design at least 5 features and report what these features capture. Report the accuracy of your classifier. Place your classifier code into the report.

In [47]:
def exercise7():
    posts = nltk.corpus.nps_chat.xml_posts()
    p = posts[:10000]

    def extract_features(post):
        features = {}
        features["suffix1"] = post.text[-1:]
        features["suffix2"] = post.text[-2:]
        features['vowels_len'] = len([word for word in post.text if word in 'aeiou'])
        features["first_letter"] = post.text[0]
        for word in nltk.word_tokenize(post.text):
            features['contains({})'.format(word.lower())] = True
        return features

    def Consecutive_classifier(posts):
        history = []
        for p in posts:
            history.append((extract_features(p), p.get('class')))
        return history

    history = Consecutive_classifier(p)
    test_size = int(len(history) * 0.1)
    train_set, test_set = history[test_size:], history[:test_size]
    classifier = nltk.NaiveBayesClassifier.train(train_set)
    print("The accuracy of consecutive classifier is- ", nltk.classify.accuracy(classifier, test_set))


exercise7()    

The accuracy of consecutive classifier is-  0.706


4.	exercise 0 (0 is a dummy ). Word features can be very useful for performing document classification, since the words that appear in a document give a strong indication about what its semantic content is. However, many words occur very infrequently, and some of the most informative words in a document may never have occurred in our training data. One solution is to make use of a lexicon, which describes how different words relate to one another. Using WordNet lexicon, augment the movie review document classifier presented in Chapter 6 to use the following two features on the intersection of words appearing in a document to classify and words appearing in “word_features”:
a)	Make a binary feature which reports “KNOWN” if the word is found in WordNet (i.e. wn.synsets is non-empty) and “UNK” if it is not found.
b)	Make a lemma name feature. Select the first synset from wn.synsets and choose the first lemma name from synset.lemma_names as the appropriate lemma. Report “UNK” if it is not found.
Report the accuracy of your classifier: use nltk.NaiveBayesClassifier, your test set should contain the first 100 instances in documents  defined as follows:
	from nltk.corpus import movie_reviews
		documents = [(list(movie_reviews.words(fileid)), category)
		     for category in movie_reviews.categories()
		     for fileid in movie_reviews.fileids(category)]
The remaining instances in documents should be part of your training set.
How does this accuracy compare to the accuracy of the classifier trained on the original feature set from the book? (Note that accuracy may not improve.) Why do you think you observe the behavior you observe? 


In [0]:
def exercise0():
  try:
    documents = [(list(movie_reviews.words(fileid)), category) for category in movie_reviews.categories() for  fileid in movie_reviews.fileids(category)]
    all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())
    word_features = list(all_words)

    def check():
      tag=[]
      for c in range(39768):
        for word_features[c] in word_features:
            if not wn.synsets(word_features[c]):
              tag.append("UNK")
            else:
              tag.append("KNOWN")
        return tag

    def check1():
      tag=[]
      for c in range(39768):
          for word in word_features:
              if not wn.synsets(word):
                  tag.append("UNK")
              else:
                  first_synsets = wn.synsets(word)[0]
                  first_lemma = first_synsets.lemma_names()
                  tag.append(first_lemma[0])
      return tag

    def document_features(document):
        features = {}
        features['binary({})'.format(word)] = check()
        features['lemma({})'.format(word)] = check1()
        return features


    featuresets = [(document_features(d),c) for (d,c) in documents]
    print("calculating")
    train_set, test_set = featuresets[100:], featuresets[:100]
    classifier = nltk.NaiveBayesClassifier.train(train_set)
    print("The accuracy of the classifier is: ",nltk.classify.accuracy(classifier, test_set))
  except ValueError:
      print("Taking long time!")
  except:
      print("Taking long time!")

exercise0()  

__Modified version:__ exercise 9.  Design at least 5 features and explain them.  Use nltk.NaiveBayesClassifier. Report the accuracy of your classifier built using all of the features that you designed.  Use show_most_inforamtive_feautures(5) functionality from the classifier to inspect the individual feature performance. Which of your features seem to be most influential? Note: http://www.nltk.org/howto/corpus.html#other-corpora provides a little more information on ppattach corpora than the textbook. Section 4 of the publication posted at https://works.bepress.com/yuliya_lierler/55/ starts by describing the dataset by Ratnaparkhi et al. (1994). This is exactly the dataset included in ppattach in NLTK.exercise 32.  Test your function on TestText with n=5, n=10, n=15, n=20, n=30. Does any of this number produce satisfactory/natural results? Include the output of your function for n=7. 

In [50]:
def exercise9():
    print('Extra Credit')
    ppattach.attachments('training')
    inst = ppattach.attachments('training')
    nattach = [inst for inst in ppattach.attachments('training') if inst.attachment == 'N']


    def get_features(inst):
        features = {}
        features['noun1'] = inst.noun1
        features['noun2'] = inst.noun2
        features['verb'] =  inst.verb
        features['sent'] = inst.sent
        features['attachment'] = inst.attachment
        return features

    feature_sets = [(get_features(inst), inst.prep) for inst in nattach]
    train_Set, test_Set = feature_sets[2200:], feature_sets[:2200]

    naiveBayesClassifier = nltk.NaiveBayesClassifier.train(train_Set)
    print('Accuracy from using NaiveBayes classifier:',nltk.classify.accuracy(naiveBayesClassifier, test_Set))
    print("")
    print(naiveBayesClassifier.most_informative_features(5))

exercise9()

Extra Credit
Accuracy from using NaiveBayes classifier: 0.5686363636363636

[('noun1', 'stake'), ('noun2', 'million'), ('verb', 'is'), ('noun1', '%'), ('noun1', 'questions')]
